In [14]:
# import libraries

import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score


In [15]:
# create a list of feature names

FEATURES = []
for i in range(1, 769):
  FEATURES.append('feature_' + str(i))


In [16]:
# create a list of label names

LABELS = ['label_1', 'label_2', 'label_3', 'label_4']


In [17]:
# read the data

df_train = pd.read_csv('/kaggle/input/layer-11-dataset/train.csv')
df_valid = pd.read_csv('/kaggle/input/layer-11-dataset/valid.csv')
df_test = pd.read_csv('/kaggle/input/layer-11-dataset/test.csv')


In [18]:
# store the data in a dictionary for each label

data_dict = dict()

for label in LABELS:
  data_dict[label] = dict()

  data_dict[label]['x_train'] = df_train[df_train[label].notna()][FEATURES].values
  data_dict[label]['y_train'] = df_train[df_train[label].notna()][label].values
  data_dict[label]['x_valid'] = df_valid[df_valid[label].notna()][FEATURES].values
  data_dict[label]['y_valid'] = df_valid[df_valid[label].notna()][label].values
  data_dict[label]['x_test'] = df_test[FEATURES].values


In [19]:
scaler = StandardScaler()

model = SVC(kernel='rbf', C=100, gamma='scale', random_state=42)

model_dict = dict()

for label in LABELS:
  model_dict[label] = SVC(kernel='rbf', C=100, gamma='scale', random_state=42)


In [20]:
# train and test the models for each label

df_test_pred = pd.DataFrame()

for label in LABELS:
  scaler.fit(data_dict[label]['x_train'])
  x_train = scaler.transform(data_dict[label]['x_train'])
  x_valid = scaler.transform(data_dict[label]['x_valid'])
  x_test = scaler.transform(data_dict[label]['x_test'])

  model_dict[label].fit(x_train, data_dict[label]['y_train'])
  data_dict[label]['y_pred'] = model_dict[label].predict(x_valid)
  print('accuracy_score for {}: '.format(label), accuracy_score(data_dict[label]['y_valid'], data_dict[label]['y_pred']))

  data_dict[label]['y_pred_test'] = model_dict[label].predict(x_test)
  df_test_pred[label] = data_dict[label]['y_pred_test']


accuracy_score for label_1:  0.952
accuracy_score for label_2:  0.9225543478260869
accuracy_score for label_3:  0.9986666666666667
accuracy_score for label_4:  0.96


In [21]:
df_test_pred.index += 1
df_test_pred.index.name = 'ID'
df_test_pred.to_csv('submission_final.csv')
